## Preamble

In [1]:
import sys

# Allows imports from the scripts directory.
sys.path.append('../scripts')

## Prepare the data

In [2]:
label2id = {
    'ISSUE': 0,
    'NONE': 1,
    'STA': 2,
    'ANALYSIS': 3,
    'PRE_RELIED': 4,
    'RATIO': 5,
    'RPC': 6,
    'PRE_NOT_RELIED': 7,
    'ARG_PETITIONER': 8,
    'PREAMBLE': 9,
    'RLC': 10,
    'ARG_RESPONDENT': 11,
    'FAC': 12
}

id2label = {v: k for k, v in label2id.items()}

In [3]:
from datautils import RRDataModule, RRBatcher

DPATH = "../data/train.json"
MPATH = "sentence-transformers/all-MiniLM-L6-v2"

batcher = RRBatcher(MPATH)
datamodule = RRDataModule(DPATH, batcher, label2id)
datamodule.setup()


c:\ProgramData\Miniconda3\envs\expt\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Prepare the model

In [4]:
from pathlib import Path
from modelling import CoherenceAwareSentenceEmbedder, AlterMiningStrategy
from pytorch_lightning.callbacks import(
    EarlyStopping,
    ModelCheckpoint,
    RichModelSummary
)

SAVE_PATH = "../models/coherence-aware/all-MiniLM-L6-v2"
CKPT = Path(SAVE_PATH).joinpath("masked-epoch=11-val_sem_loss=0.29.ckpt")


surrogate_imp=0.2
try:
    model = CoherenceAwareSentenceEmbedder.load_from_checkpoint(CKPT, num_classes=len(label2id))
    print("Loaded checkpoint")
except:
    model = CoherenceAwareSentenceEmbedder(MPATH,  num_classes=len(label2id), surrogate_imp=surrogate_imp, triplet_margin=1.0)
    print("Loaded fresh model")

checkpoint_callback = ModelCheckpoint(
    SAVE_PATH,
    filename=f'imp={surrogate_imp}-{{epoch}}-{{val_sem_loss:.2f}}',
    monitor='val_sem_loss',
    # save_weights_only=True,
)
early_stop_callback = EarlyStopping(
    monitor="val_sem_loss",
    min_delta=1e-4, patience=8,
    verbose=False,
    mode="min"
)

mining_callback = AlterMiningStrategy(monitor="val_sem_loss")

Loaded fresh model


## Train the model

In [5]:
from pytorch_lightning import Trainer, seed_everything


seed_everything(42, workers=True)

trainer = Trainer(
    max_epochs=-1,
    deterministic=True,
    accumulate_grad_batches=4,
    callbacks=[checkpoint_callback, early_stop_callback, RichModelSummary(), mining_callback],
    accelerator='gpu',
    log_every_n_steps=16
)

trainer.fit(model, datamodule)

Global seed set to 42
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\ProgramData\Miniconda3\envs\expt\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\repos\sem-eval23\models\coherence-aware\all-MiniLM-L6-v2 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                                      ┃ Type               ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ backbone                                  │ BertModel          │ 22.7 M │
│ 1  │ backbone.embeddings                       │ BertEmbeddings     │ 11.9 M │
│ 2  │ backbone.embeddings.word_embeddings       │ Embedding          │ 11.7 M │
│ 3  │ backbone.embeddings.position_embeddings   │ Embedding          │  196 K │
│ 4  │ backbone.embeddings.token_type_embeddings │ Embedding          │    768 │
│ 5  │ backbone.embeddings.LayerNorm             │ LayerNorm          │    768 │
│ 6  │ backbone.embeddings.dropout               │ Dropout            │      0 │
│ 7  │ backbone.encoder                          │ BertEncoder        │ 10.6 M │
│ 8  │ backbone.encoder.layer                    │ ModuleList         │ 10.6 M │
│ 9  │ backbone.encoder.layer.0                  │ BertLayer          │  1.8 M │
│ 10 │ backbone.encoder.layer.0.attention        │ BertAttention      │  592 K │
│ 11 │ backbone.encoder.layer.0.intermediate     │ BertIntermediate   │  591 K │
│ 12 │ backbone.encoder.layer.0.output           │ BertOutput         │  590 K │
│ 13 │ backbone.encoder.layer.1                  │ BertLayer          │  1.8 M │
│ 14 │ backbone.encoder.layer.1.attention        │ BertAttention      │  592 K │
│ 15 │ backbone.encoder.layer.1.intermediate     │ BertIntermediate   │  591 K │
│ 16 │ backbone.encoder.layer.1.output           │ BertOutput         │  590 K │
│ 17 │ backbone.encoder.layer.2                  │ BertLayer          │  1.8 M │
│ 18 │ backbone.encoder.layer.2.attention        │ BertAttention      │  592 K │
│ 19 │ backbone.encoder.layer.2.intermediate     │ BertIntermediate   │  591 K │
│ 20 │ backbone.encoder.layer.2.output           │ BertOutput         │  590 K │
│ 21 │ backbone.encoder.layer.3                  │ BertLayer          │  1.8 M │
│ 22 │ backbone.encoder.layer.3.attention        │ BertAttention      │  592 K │
│ 23 │ backbone.encoder.layer.3.intermediate     │ BertIntermediate   │  591 K │
│ 24 │ backbone.encoder.layer.3.output           │ BertOutput         │  590 K │
│ 25 │ backbone.encoder.layer.4                  │ BertLayer          │  1.8 M │
│ 26 │ backbone.encoder.layer.4.attention        │ BertAttention      │  592 K │
│ 27 │ backbone.encoder.layer.4.intermediate     │ BertIntermediate   │  591 K │
│ 28 │ backbone.encoder.layer.4.output           │ BertOutput         │  590 K │
│ 29 │ backbone.encoder.layer.5                  │ BertLayer          │  1.8 M │
│ 30 │ backbone.encoder.layer.5.attention        │ BertAttention      │  592 K │
│ 31 │ backbone.encoder.layer.5.intermediate     │ BertIntermediate   │  591 K │
│ 32 │ backbone.encoder.layer.5.output           │ BertOutput         │  590 K │
│ 33 │ backbone.pooler                           │ BertPooler         │  147 K │
│ 34 │ backbone.pooler.dense                     │ Linear             │  147 K │
│ 35 │ backbone.pooler.activation                │ Tanh               │      0 │
│ 36 │ pooling                                   │ Pooling            │      0 │
│ 37 │ sem_infuser                               │ SemanticInfuser    │  1.0 K │
│ 38 │ sem_infuser.infuser                       │ Conv1d             │  1.0 K │
│ 39 │ sem_infuser.presenter                     │ Conv1d             │      5 │
│ 40 │ surrogate_head                            │ Linear             │    258 │
│ 41 │ miner                                     │ BatchEasyHardMiner │      0 │
│ 42 │ miner.distance                            │ LpDistance         │      0 │
│ 43 │ surr_loss                                 │ BCEWithLogitsLoss  │      0 │
│ 44 │ sem_loss                                  │ TripletMarginLoss  │      0 │
│ 45 │ sem_loss.distance                         │ LpDistance         │      0 │
│ 46 │ sem_loss.reducer        

Trainable params: 1.3 K                                                                                            
Non-trainable params: 22.7 M                                                                                       
Total params: 22.7 M                                                                                               
Total estimated model params size (MB): 90

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\ProgramData\Miniconda3\envs\expt\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


c:\ProgramData\Miniconda3\envs\expt\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 1:  96%|█████████▌| 219/228 [00:31<00:01,  6.93it/s, loss=0.752, v_num=9, train_loss=0.729, train_sem_loss=0.838, train_surr_loss=0.293, val_loss=0.769, val_sem_loss=0.863, val_surr_loss=0.390]

c:\ProgramData\Miniconda3\envs\expt\lib\site-packages\pytorch_lightning\trainer\trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
